<a href="https://colab.research.google.com/github/raphaelp-silva/deep_learning_com_pytorch_e_python/blob/main/Projeto_3_Classifica%C3%A7%C3%A3o_bin%C3%A1ria_breast_cancer_com_tuning_de_par%C3%A2metros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto 3: Classificação binária brest cancer com tuning dos parâmetros

## Etapa 1: Importação das bibliotecas

In [2]:
!pip install skorch

In [3]:
import pandas as pd
import numpy as np
import sklearn
import skorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import GridSearchCV
from skorch import NeuralNetBinaryClassifier

In [4]:
torch.__version__, skorch.__version__, sklearn.__version__

('2.5.1+cu124', '1.1.0', '1.6.1')

## Etapa 2: Base de dados

In [5]:
np.random.seed(123)
torch.manual_seed(123)

In [6]:
previsores = pd.read_csv('entradas_breast.csv')
classe = pd.read_csv('saidas_breast.csv')

In [7]:
previsores = np.array(previsores, dtype = 'float32')
classe = np.array(classe, dtype = 'float32').squeeze(1)

In [8]:
previsores.shape

(569, 30)

In [9]:
classe.shape

(569,)

## Etapa 3: Classe para estrutura da rede neural

**\*\* ATUALIZAÇÃO JAN/2022 \*\*** : na versão atual do Skorch, os resultados da rede neural devem ser retornados sem ativação, ou seja, sem a camada sigmoide no final. Com isto, a função de custo deve ser `BCEWithLogitsLoss`.

In [10]:
class classificador_torch(nn.Module):
  def __init__(self, activation, neurons, initializer):
    super().__init__()

    # 30 -> 16 -> 16 -> 1  // estrutura da rede neural
    self.dense0 = nn.Linear(30, neurons)
    initializer(self.dense0.weight)
    self.activation0 = activation

    self.dense1 = nn.Linear(neurons, neurons)
    initializer(self.dense1.weight)
    self.activation1 = activation

    self.dense2 = nn.Linear(neurons, 1)
    initializer(self.dense2.weight)
    self.output_activation = nn.Sigmoid()

  def forward(self, x):
    x = self.dense0(x)
    x = self.activation0(x)
    x = self.dense1(x)
    x = self.activation1(x)
    x = self.dense2(x)
    x = self.output_activation(x)
    return x

## Etapa 4: Skorch

In [11]:
classificador_sklearn = NeuralNetBinaryClassifier(module = classificador_torch,
                                                  lr = 0.001,
                                                  optimizer__weight_decay = 0.0001,
                                                  train_split=False)

## Etapa 5: Tuning dos parâmetros

In [12]:
# definindo os parâmetros que serão testados na rede para encontrar o que melhor perfomou nos testes:
# O ideal é ir testando parâmetro por parâmetro e não todos de uma vez só.

params = {'batch_size' : [10,30],
          'max_epochs' : [50,100],
          'optimizer' : [torch.optim.Adam, torch.optim.SGD],
          'criterion' : [torch.nn.BCELoss, torch.nn.HingeEmbeddingLoss],
          'module__activation' : [F.relu, F.tanh],
          'module__neurons' : [8, 16],
          'module__initializer' : [torch.nn.init.uniform_, torch.nn.init.normal_]}

In [13]:
params

{'batch_size': [10, 30],
 'max_epochs': [50, 100],
 'optimizer': [torch.optim.adam.Adam, torch.optim.sgd.SGD],
 'criterion': [torch.nn.modules.loss.BCELoss,
  torch.nn.modules.loss.HingeEmbeddingLoss],
 'module__activation': [<function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
  <function torch.nn.functional.tanh(input)>],
 'module__neurons': [8, 16],
 'module__initializer': [<function torch.nn.init.uniform_(tensor: torch.Tensor, a: float = 0.0, b: float = 1.0, generator: Optional[torch._C.Generator] = None) -> torch.Tensor>,
  <function torch.nn.init.normal_(tensor: torch.Tensor, mean: float = 0.0, std: float = 1.0, generator: Optional[torch._C.Generator] = None) -> torch.Tensor>]}

In [14]:
grid_search = GridSearchCV(estimator = classificador_sklearn, param_grid = params,
                           scoring = 'accuracy',
                           cv = 2)
grid_search = grid_search.fit(previsores, classe)

  epoch    train_loss     dur
-------  ------------  ------
      1       37.3239  0.0599
      2       37.3239  0.0397
      3       37.3239  0.0436
      4       37.3239  0.0378
      5       37.3239  0.0365
      6       37.3239  0.0371
      7       37.3239  0.0406
      8       37.3239  0.0401
      9       37.3239  0.0517
     10       37.3239  0.0541
     11       37.3239  0.0568
     12       37.3239  0.0540
     13       37.3239  0.0540
     14       37.3239  0.0576
     15       37.3239  0.0527
     16       37.3239  0.0505
     17       37.3239  0.0479
     18       37.3239  0.0516
     19       37.3239  0.0519
     20       37.3239  0.0625
     21       37.3239  0.0758
     22       37.3239  0.0707
     23       37.3239  0.0496
     24       37.3239  0.0482
     25       37.3239  0.0504
     26       37.3239  0.0495
     27       37.3239  0.0477
     28       37.3239  0.0463
     29       37.3239  0.0521
     30       37.3239  0.0489
     31       37.3239  0.0494
     32   

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 380, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 90, in _cached_call
   

      5        1.0000  0.0474
      6        1.0000  0.0558
      7        1.0000  0.0540
      8        1.0000  0.0503
      9        1.0000  0.0436
     10        1.0000  0.0441
     11        1.0000  0.0437
     12        1.0000  0.0406
     13        1.0000  0.0434
     14        1.0000  0.0444
     15        1.0000  0.0472
     16        1.0000  0.0415
     17        1.0000  0.0421
     18        1.0000  0.0431
     19        1.0000  0.0415
     20        1.0000  0.0453
     21        1.0000  0.0437
     22        1.0000  0.0432
     23        1.0000  0.0416
     24        1.0000  0.0428
     25        1.0000  0.0426
     26        1.0000  0.0442
     27        1.0000  0.0500
     28        1.0000  0.0574
     29        1.0000  0.0526
     30        1.0000  0.0454
     31        1.0000  0.0429
     32        1.0000  0.0423
     33        1.0000  0.0433
     34        1.0000  0.0426
     35        1.0000  0.0426
     36        1.0000  0.0422
     37        1.0000  0.0428
     38   

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 380, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 90, in _cached_call
   

      6        1.0000  0.0371
      7        1.0000  0.0323
      8        1.0000  0.0328
      9        1.0000  0.0332
     10        1.0000  0.0317
     11        1.0000  0.0326
     12        1.0000  0.0320
     13        1.0000  0.0322
     14        1.0000  0.0338
     15        1.0000  0.0308
     16        1.0000  0.0319
     17        1.0000  0.0316
     18        1.0000  0.0319
     19        1.0000  0.0320
     20        1.0000  0.0383
     21        1.0000  0.0344
     22        1.0000  0.0326
     23        1.0000  0.0319
     24        1.0000  0.0319
     25        1.0000  0.0311
     26        1.0000  0.0333
     27        1.0000  0.0298
     28        1.0000  0.0412
     29        1.0000  0.0392
     30        1.0000  0.0481
     31        1.0000  0.0344
     32        1.0000  0.0321
     33        1.0000  0.0308
     34        1.0000  0.0325
     35        1.0000  0.0311
     36        1.0000  0.0314
     37        1.0000  0.0323
     38        1.0000  0.0316
     39   

...

     11        0.4444  0.0170
     12        0.4443  0.0166
     13        0.4442  0.0162
     14        0.4441  0.0166
     15        0.4439  0.0160
     16        0.4438  0.0168
     17        0.4437  0.0166
     18        0.4436  0.0168
     19        0.4435  0.0164
     20        0.4434  0.0168
     21        0.4433  0.0193
     22        0.4432  0.0184
     23        0.4431  0.0192
     24        0.4430  0.0221
     25        0.4429  0.0167
     26        0.4428  0.0163
     27        0.4427  0.0160
     28        0.4426  0.0174
     29        0.4425  0.0163
     30        0.4424  0.0191
     31        0.4422  0.0190
     32        0.4421  0.0186
     33        0.4420  0.0192
     34        0.4419  0.0196
     35        0.4418  0.0195
     36        0.4417  0.0195
     37        0.4416  0.0188
     38        0.4415  0.0163
     39        0.4414  0.0186
     40        0.4413  0.0193
     41        0.4412  0.0195
     42        0.4411  0.0194
     43        0.4410  0.0192
     44   

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 380, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 90, in _cached_call
   

     11        0.4633  0.0172
     12        0.4631  0.0183
     13        0.4630  0.0165
     14        0.4628  0.0168
     15        0.4627  0.0161
     16        0.4626  0.0163
     17        0.4624  0.0162
     18        0.4623  0.0161
     19        0.4621  0.0163
     20        0.4620  0.0167
     21        0.4618  0.0162
     22        0.4616  0.0173
     23        0.4615  0.0164
     24        0.4614  0.0174
     25        0.4612  0.0178
     26        0.4611  0.0165
     27        0.4609  0.0233
     28        0.4608  0.0274
     29        0.4606  0.0246
     30        0.4605  0.0181
     31        0.4603  0.0174
     32        0.4602  0.0191
     33        0.4601  0.0197
     34        0.4600  0.0195
     35        0.4598  0.0191
     36        0.4596  0.0199
     37        0.4595  0.0196
     38        0.4594  0.0191
     39        0.4593  0.0196
     40        0.4591  0.0165
     41        0.4590  0.0164
     42        0.4588  0.0169
     43        0.4587  0.0168
     44   

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:960: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 949, in _score
    scores = scorer(estimator, X_test, y_test, **score_params)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 288, in __call__
    return self._score(partial(_cached_call, None), estimator, X, y_true, **_kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 380, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_scorer.py", line 90, in _cached_call
   

      2       37.2583  0.1173
      3       37.2583  0.1081
      4       37.2583  0.1168
      5       37.2583  0.1234
      6       37.2583  0.1268
      7       37.2583  0.1347
      8       37.2583  0.1126
      9       37.2583  0.1017
     10       37.2583  0.1048
     11       37.2583  0.1009
     12       37.2583  0.1080
     13       37.2583  0.1118
     14       37.2583  0.0970
     15       37.2583  0.1103
     16       37.2583  0.1089
     17       37.2583  0.0961
     18       37.2583  0.0972
     19       25.6044  0.0995
     20        0.7398  0.0991
     21        0.5741  0.1012
     22        0.5199  0.1030
     23        0.5026  0.1075
     24        0.4894  0.1013
     25        0.4748  0.1050
     26        0.4741  0.1298
     27        0.4441  0.1227
     28        0.4329  0.1066
     29        0.4226  0.1018
     30        0.4099  0.1180
     31        0.3963  0.1345
     32        0.3858  0.1130
     33        0.3781  0.1334
     34        0.3693  0.1340
     35   

In [15]:
melhores_parametros = grid_search.best_params_
melhor_precisao = grid_search.best_score_

In [16]:
melhores_parametros

{'batch_size': 10,
 'criterion': torch.nn.modules.loss.BCELoss,
 'max_epochs': 100,
 'module__activation': <function torch.nn.functional.relu(input: torch.Tensor, inplace: bool = False) -> torch.Tensor>,
 'module__initializer': <function torch.nn.init.uniform_(tensor: torch.Tensor, a: float = 0.0, b: float = 1.0, generator: Optional[torch._C.Generator] = None) -> torch.Tensor>,
 'module__neurons': 8,
 'optimizer': torch.optim.adam.Adam}

In [17]:
melhor_precisao

0.7594576229305658